In [9]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2025 NVIDIA Corporation
Built on Fri_Feb_21_20:23:50_PST_2025
Cuda compilation tools, release 12.8, V12.8.93
Build cuda_12.8.r12.8/compiler.35583870_0


In [10]:
!python --version

Python 3.10.12


In [11]:
print("Installing libraries...")
!CMAKE_ARGS="-DGGML_CUDA=on" pip install llama-cpp-python --quiet

Installing libraries...


In [14]:
%pip install errant huggingface-hub --quiet
!python -m spacy download en_core_web_sm --quiet

Note: you may need to restart the kernel to use updated packages.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [16]:
from llama_cpp import Llama
import os
import urllib.request
import subprocess
import tempfile
import re
import time


In [17]:
# order matters: always substitute the longer N'T first, then the shorter endings
CONTRACTIONS = ["n't", "'ll", "'ve", "'re", "'d", "'m", "'s"]

# characters that carry a *leading* space in BEA tokenisation
PUNCT = r"\.\,\!\?\:\;"

def detokenize_bea(text: str) -> str:
    """
    Convert BEA‑style tokenisation to normal English prose.
    """
    text = text.replace("’", "'")
    # 1. Join contractions
    #   e.g. "do n't"  -> "don't"
    #        "she 'll" -> "she'll"
    # pattern:    <word> SP <contr>
    for contr in CONTRACTIONS:
        pattern = rf"\b(\w+)\s+{re.escape(contr)}\b"
        repl    = rf"\1{contr}"
        text    = re.sub(pattern, repl, text)

    # 2. Collapse space BEFORE punctuation  ( ... "word ." -> "word." )
    #  – only collapse if it’s a *single* normal blank, keep new‑lines
    text = re.sub(r" (?=[{}])".format(PUNCT), "", text)

    # 3. Clean spaces just inside brackets  : "( word" -> "(word"
    text = re.sub(r"\(\s+", "(", text)
    text = re.sub(r"\[\s+", "[", text)
    text = re.sub(r"\s+\)", ")", text)
    text = re.sub(r"\s+\]", "]", text)

    # 4. Collapse 2+ blanks into one
    text = re.sub(r"[ \t]{2,}", " ", text)

    return text


def tokenize_like_bea(text: str) -> str:
    """
    Convert normal English prose back to the peculiar BEA tokenisation.
    (Not 100 % identical for every imaginable corner‑case, but round‑trips
     correctly for the typical patterns in the dataset.)
    """
    text = text.replace("’", "'")
    # 1. Split contractions      e.g. "don't" -> "do n't"
    # n't must be done first because "won't" → "wo n't", not "wo n' t"
    text = re.sub(r"\b(\w+)n't\b",  r"\1 n't", text)

    # remaining endings
    for contr in ["'ll", "'ve", "'re", "'d", "'m", "'s"]:
        pattern = rf"\b(\w+){re.escape(contr)}\b"
        repl    = rf"\1 {contr}"
        text    = re.sub(pattern, repl, text)

    # 2. Add a space *before* the main punctuation marks
    # We first remove any existing spaces, then add exactly one.
    text = re.sub(r"\s*([{}])".format(PUNCT), r" \1", text)

    # 3. Remove extra double spaces that might have appeared
    text = re.sub(r"[ \t]{2,}", " ", text)

    return text



In [18]:

# --- Configuration ---
MAX_EVAL_SAMPLES = 10 # Set to None to evaluate on the full dataset
LOG_SAMPLE_OUTPUTS = 10 # Number of original/corrected pairs to print per model
# GEC_PROOMPT = "Fix grammaticality in this sentence:"
GEC_PROOMPT = ""

# --- Loading data ---

# Raw URL of the file
bea_dev = "https://raw.githubusercontent.com/grammarly/pillars-of-gec/main/data/evaluation_sets/bea-dev.txt"
bea_m2 = "https://raw.githubusercontent.com/grammarly/pillars-of-gec/refs/heads/main/data/evaluation_sets/bea-dev.m2"

folder = "data"
os.makedirs(folder, exist_ok=True)

dev_file_path = os.path.join(folder, "bea-dev.txt")
if not os.path.exists(dev_file_path):
    print(f"Downloading {bea_dev}...")
    urllib.request.urlretrieve(bea_dev, dev_file_path)
    print(f"File downloaded to: {dev_file_path}")
else:
    print(f"Using existing file: {dev_file_path}")


m2_file_path = os.path.join(folder, "bea-dev.m2")
if not os.path.exists(m2_file_path):
    print(f"Downloading {bea_m2}...")
    urllib.request.urlretrieve(bea_m2, m2_file_path)
    print(f"File downloaded to: {m2_file_path}")
else:
     print(f"Using existing file: {m2_file_path}")


# --- Utility Functions ---
def read_sentences(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f]

def create_subset_m2(input_m2_path, output_m2_path, num_sentences):
    """Reads an M2 file and writes the first num_sentences entries to a new file."""
    print(f"Creating subset M2 file ({num_sentences} sentences) at: {output_m2_path}")
    sentence_count = 0
    with open(input_m2_path, 'r', encoding='utf-8') as infile, \
         open(output_m2_path, 'w', encoding='utf-8') as outfile:
        for line in infile:
            # Ensure line is not empty before checking prefix
            if line.strip() and line.startswith('S '):
                sentence_count += 1
                if sentence_count > num_sentences:
                    break
            # Check sentence_count before writing any line related to the current sentence block
            if sentence_count <= num_sentences and sentence_count > 0: # Only write if we are within the limit and started processing sentences
                 outfile.write(line)
            # Handle edge case where the loop might end before writing the last newline of the final sentence block
        if sentence_count <= num_sentences and sentence_count > 0:
             outfile.write('\n') # Ensure last entry ends properly if needed

    print(f"Finished creating subset M2 file.")


# --- Read Original Sentences (BEA Format) ---
all_original_sentences_bea = read_sentences(dev_file_path)
print(f"Read {len(all_original_sentences_bea)} total sentences from {dev_file_path}")

# Select subset for evaluation
if MAX_EVAL_SAMPLES is not None and MAX_EVAL_SAMPLES < len(all_original_sentences_bea):
    original_sentences_bea = all_original_sentences_bea[:MAX_EVAL_SAMPLES]
    print(f"Using a subset of {len(original_sentences_bea)} sentences for evaluation.")
else:
    original_sentences_bea = all_original_sentences_bea
    print(f"Using the full dataset of {len(original_sentences_bea)} sentences for evaluation.")



File downloaded to: data/bea-dev.txt
File downloaded to: data/bea-dev.m2
Read 4384 total sentences from data/bea-dev.txt
Using a subset of 10 sentences for evaluation.


In [20]:
class LLM:
    def __init__(self, repo_id: str, filename: str, verbose: bool = False, n_gpu_layers=-1, gec_prompt:str = GEC_PROOMPT, **kwargs):
        self.model_name = repo_id.split('/')[-1] # Extract model name for reporting
        print(f"Initializing {self.model_name}...")
        self.gec_prompt = gec_prompt
        print(f"Using gec prompt: {gec_prompt}")
        self.model = Llama.from_pretrained(
            repo_id=repo_id,
            filename=filename,
            verbose=verbose,
            n_gpu_layers=n_gpu_layers,
            **kwargs
        )
        print(f"{self.model_name} initialized.")

    # Takes NATURAL language prompts, returns NATURAL language results
    def __call__(self, prompts: list[str], system_prompt: str = "Correct *all* grammatical, punctuation, and spelling errors. Only output the corrected sentence.", temperature: float = 0.2, max_tokens=200, **kwargs):
        results = []
        total_prompts = len(prompts)
        for i, prompt in enumerate(prompts):
            # Add progress indicator
            print(f"  Processing prompt {i+1}/{total_prompts}...", end='\r')
            try:
                response = self.model.create_chat_completion(
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": f"{self.gec_prompt}+ {prompt}"}
                    ],
                    temperature=temperature,
                    max_tokens=max_tokens,
                    **kwargs
                )
                # Handle potential empty responses or formatting issues
                if response['choices'] and 'message' in response['choices'][0] and 'content' in response['choices'][0]['message']:
                    correction = response['choices'][0]['message']['content'].strip()
                    # Basic check for empty string, could be more robust
                    if correction:
                         results.append(correction)
                    else:
                        print(f"\nWarning: Model generated empty output for prompt: {prompt[:80]}...") # Log snippet
                        results.append(prompt) # Use original sentence as fallback
                else:
                    print(f"\nWarning: Unexpected response structure or no choices for prompt: {prompt[:80]}...")
                    results.append(prompt) # Fallback to original
            except Exception as e:
                print(f"\nError during model inference for prompt: {prompt[:80]}...\n{e}")
                # Fallback to original sentence on error to avoid crashing evaluation
                results.append(prompt)
        print(f"\n  Finished processing {total_prompts} prompts.") # Newline after progress
        return results


In [22]:
# --- Model Initialization ---
# Define configurations instead of loading immediately
model_configs = {
    "gemma-3-4b-it": {
        "repo_id": "unsloth/gemma-3-4b-it-GGUF",
        "filename": "*Q4_K_M.gguf",
        "verbose": False,
        "gec_prompt": GEC_PROOMPT # Use the global GEC_PROOMPT
    },
    "nomodit-4b-v0": {
        "repo_id": "muzzz/nomodit-4b-merged-v0-Q4_K_M-GGUF",
        "filename": "*q4_k_m.gguf",
        "verbose": False,
        "gec_prompt": "Fix grammaticality in this sentence:" # Specific prompt for this model
    },
    "nomodit-4b": {
        "repo_id": "muzzz/nomodit-4b-merged-Q4_K_M-GGUF",
        "filename": "*q4_k_m.gguf",
        "verbose": False,
        "gec_prompt": "Fix grammaticality in this sentence:" # Specific prompt for this model
    },
    # Add configurations for commented-out models if you plan to use them later
    # "phi-4-mini-instruct": {"repo_id": "unsloth/Phi-4-mini-instruct-GGUF", "filename": "*Q4_K_M.gguf", "verbose": False, "gec_prompt": GEC_PROOMPT},
    # "phi-4": {"repo_id": "unsloth/phi-4-GGUF", "filename": "*Q4_K_M.gguf", "verbose": False, "gec_prompt": GEC_PROOMPT},
    # "llama-3.1-8B-Instruct": {"repo_id": "bartowski/Meta-Llama-3.1-8B-Instruct-GGUF", "filename": "*Q4_K_M.gguf", "verbose": False, "gec_prompt": GEC_PROOMPT},
    # "llama-3.2-3B-Instruct": {"repo_id": "unsloth/Llama-3.2-3B-Instruct-GGUF", "filename": "*Q4_K_M.gguf", "verbose": False, "gec_prompt": GEC_PROOMPT},
}


In [23]:
# --- Benchmarking ---

models_to_benchmark_names = [
    "gemma-3-4b-it",
    "nomodit-4b-v0",
    "nomodit-4b",
    # "phi-4-mini-instruct",
    # "phi-4",
    # "llama-3.1-8B-Instruct",
    # "llama-3.2-3B-Instruct",
]

results = {}

# Regex to capture P, R, F0.5 from errant_compare output
# Updated regex to handle the table format output
score_pattern = re.compile(
    r"=========== Span-Based Correction ============\n"  # Match header
    r"TP\s+FP\s+FN\s+Prec\s+Rec\s+F0\.5\n"             # Match table header row
    r"\d+\s+\d+\s+\d+\s+(\d+\.\d+)\s+(\d+\.\d+)\s+(\d+\.\d+)" # Capture Prec, Rec, F0.5 from data row
)



In [24]:

print("\n--- Starting Benchmark ---")

# --- Prepare Input Data --- #
# Detokenize the input sentences ONCE before the loop
print(f"Detokenizing {len(original_sentences_bea)} sentences for LLM input...")
natural_input_sentences = [detokenize_bea(s) for s in original_sentences_bea]
print("Detokenization complete.")

# --- Create Reference M2 Subset (if needed) --- #
reference_m2_to_use = m2_file_path
temp_ref_m2_subset_path = None # Initialize path variable

if MAX_EVAL_SAMPLES is not None and MAX_EVAL_SAMPLES < len(all_original_sentences_bea):
    # Create a temporary file path that persists until explicitly deleted
    temp_ref_m2_file = tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".m2", encoding='utf-8')
    temp_ref_m2_subset_path = temp_ref_m2_file.name
    temp_ref_m2_file.close() # Close the file handle immediately, but keep the file
    create_subset_m2(m2_file_path, temp_ref_m2_subset_path, len(original_sentences_bea))
    reference_m2_to_use = temp_ref_m2_subset_path
    print(f"Using subset reference M2: {reference_m2_to_use}")
else:
    print(f"Using full reference M2: {reference_m2_to_use}")


for model_key in models_to_benchmark_names:
    config = model_configs.get(model_key)
    if not config:
        print(f"Warning: Model key '{model_key}' not found in model configurations. Skipping.")
        continue

    # Initialize LLM object here, inside the loop
    print(f"\n===== Loading model: {model_key} =====\n")
    llm = None # Initialize llm to None
    try:
        llm = LLM(**config) # Load the model using its specific config
        model_name = llm.model_name # Use name from LLM object
        print(f"\n===== Evaluating model: {model_name} =====")

        # Paths for temporary files specific to this model iteration
        temp_orig_bea_path = None
        temp_cor_bea_path = None
        temp_hyp_m2_path = None

        # --- Create temporary file for ORIGINAL sentences (BEA format) --- #
        # This file is needed as -orig input for errant_parallel
        with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".txt", encoding='utf-8') as temp_orig_file:
            for sentence in original_sentences_bea: # Write the BEA format sentences
                temp_orig_file.write(sentence + '\n')
            temp_orig_bea_path = temp_orig_file.name

        # --- Run Model (using detokenized input) --- #
        start_time = time.time()
        print(f"Generating corrections for {len(natural_input_sentences)} sentences...")
        # Pass the NATURAL language sentences to the LLM
        corrected_sentences_natural = llm(natural_input_sentences)
        generation_time = time.time() - start_time
        print(f"Generation took: {generation_time:.2f} seconds")

        # --- Retokenize Output --- #
        print("Retokenizing LLM output to BEA format...")
        corrected_sentences_bea = [tokenize_like_bea(s) for s in corrected_sentences_natural]
        print("Retokenization complete.")


        # --- Log Sample Outputs --- #
        if LOG_SAMPLE_OUTPUTS > 0:
            print(f"\n--- Sample Outputs for {model_name} (First {min(LOG_SAMPLE_OUTPUTS, len(original_sentences_bea))}) ---")
            for i in range(min(LOG_SAMPLE_OUTPUTS, len(original_sentences_bea))):
                print(f"Orig (BEA)  [{i+1}]: {original_sentences_bea[i]}")
                # print(f"Input (Nat) [{i+1}]: {natural_input_sentences[i]}") # Optional: print detokenized input
                # print(f"Output (Nat)[{i+1}]: {corrected_sentences_natural[i]}") # Optional: print natural output
                print(f"Corr (BEA)  [{i+1}]: {corrected_sentences_bea[i]}")
                print("---")
            print("\n")

        # --- Evaluation --- #
        if len(corrected_sentences_bea) != len(original_sentences_bea):
            print(f"Error: Number of corrected sentences ({len(corrected_sentences_bea)}) does not match original ({len(original_sentences_bea)}) for model {model_name}. Skipping evaluation.")
            results[model_name] = {"P": "Error", "R": "Error", "F0.5": "Length Mismatch", "Generation Time (s)": f"{generation_time:.2f}", "Eval Time (s)": "N/A"}
            # No 'continue' here, need to go to finally block

        else: # Only run eval if lengths match
            eval_start_time = time.time()

            # 3. Write RETOKENIZED CORRECTED sentences (BEA format) to a temporary file
            with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".cor.txt", encoding='utf-8') as temp_cor_file:
                for sentence in corrected_sentences_bea: # Write the BEA format sentences
                    temp_cor_file.write(sentence + '\n')
                temp_cor_bea_path = temp_cor_file.name
            print(f"Created temp corrected (BEA format): {temp_cor_bea_path}")


            # 4. Create hypothesis M2 file using errant_parallel
            #    Input: -orig (BEA format), -cor (BEA format)
            #    Output: hypothesis M2 (ERRANT will likely re-tokenize internally based on spaCy,
            #            but we feed it BEA format for consistency at this step)
            with tempfile.NamedTemporaryFile(mode='w', delete=False, suffix=".hyp.m2") as temp_hyp_m2_file:
                temp_hyp_m2_path = temp_hyp_m2_file.name

            print(f"Running errant_parallel... Orig: {temp_orig_bea_path}, Cor: {temp_cor_bea_path}, Out: {temp_hyp_m2_path}")
            parallel_cmd = ["errant_parallel", "-orig", temp_orig_bea_path, "-cor", temp_cor_bea_path, "-out", temp_hyp_m2_path]
            parallel_result = subprocess.run(parallel_cmd, capture_output=True, text=True, check=False)

            if parallel_result.returncode != 0:
                print(f"Error running errant_parallel for {model_name}:")
                print(f"Stderr: {parallel_result.stderr}")
                results[model_name] = {"P": "Error", "R": "Error", "F0.5": "errant_parallel failed", "Generation Time (s)": f"{generation_time:.2f}", "Eval Time (s)": "N/A"}
                # No 'continue' here, need to go to finally block

            elif "Processing 0 sentences" in parallel_result.stderr: # Check if ERRANT processed anything
                 print(f"Warning: errant_parallel reported processing 0 sentences. Check input files or ERRANT setup.")
                 print(f"Stderr: {parallel_result.stderr}")
                 # Decide whether to continue or mark as error - currently just warns
                 results[model_name] = {"P": "N/A", "R": "N/A", "F0.5": "ERRANT processed 0", "Generation Time (s)": f"{generation_time:.2f}", "Eval Time (s)": "N/A"}

            else:
                # 5. Compare hypothesis M2 with reference M2 using errant_compare
                #    Input: -hyp (Generated M2), -ref (BEA Reference M2 - potentially subsetted)
                print(f"Running errant_compare... Hyp: {temp_hyp_m2_path}, Ref: {reference_m2_to_use}")
                compare_cmd = ["errant_compare", "-hyp", temp_hyp_m2_path, "-ref", reference_m2_to_use]
                compare_result = subprocess.run(compare_cmd, capture_output=True, text=True, check=False)

                if compare_result.returncode != 0:
                    print(f"Error running errant_compare for {model_name}:")
                    print(f"Stderr: {compare_result.stderr}")
                    results[model_name] = {"P": "Error", "R": "Error", "F0.5": "errant_compare failed", "Generation Time (s)": f"{generation_time:.2f}", "Eval Time (s)": "N/A"}
                    # No 'continue' here, need to go to finally block
                else:
                    # 6. Parse scores
                    eval_time = time.time() - eval_start_time
                    print(f"Evaluation took: {eval_time:.2f} seconds")
                    # print(f"errant_compare output for {model_name}:\n{compare_result.stdout}") # Uncomment for debugging

                    match = score_pattern.search(compare_result.stdout)
                    if match:
                        p, r, f05 = match.groups()
                        print(f"Scores for {model_name} (on {len(original_sentences_bea)} samples): P={p}, R={r}, F0.5={f05}")
                        results[model_name] = {
                            "P": float(p),
                            "R": float(r),
                            "F0.5": float(f05),
                            "Samples": len(original_sentences_bea),
                            "Generation Time (s)": f"{generation_time:.2f}",
                            "Eval Time (s)": f"{eval_time:.2f}"
                        }
                    else:
                        print(f"Could not parse scores from errant_compare output for {model_name}.")
                        print(f"Output:\n{compare_result.stdout}")
                        results[model_name] = {"P": "Error", "R": "Error", "F0.5": "Parsing failed", "Samples": len(original_sentences_bea), "Generation Time (s)": f"{generation_time:.2f}", "Eval Time (s)": f"{eval_time:.2f}"}

    except Exception as e:
        print(f"An unexpected error occurred during loading or evaluation for {model_key}: {e}")
        # Use model_key if llm object wasn't successfully created
        current_model_name = llm.model_name if llm else model_key
        gen_time_str = f"{generation_time:.2f}" if 'generation_time' in locals() else "N/A"
        eval_time_str = f"{eval_time:.2f}" if 'eval_time' in locals() else "N/A"

        if 'eval_start_time' in locals():
             current_eval_time = time.time() - eval_start_time
             eval_time_str = f"{current_eval_time:.2f}"
        elif 'start_time' in locals():
             current_gen_time = time.time() - start_time
             gen_time_str = f"{current_gen_time:.2f}"


        results[current_model_name] = {
             "P": "Error", "R": "Error", "F0.5": f"Exception: {type(e).__name__}",
             "Samples": len(original_sentences_bea) if 'original_sentences_bea' in locals() else 'N/A',
             "Generation Time (s)": gen_time_str,
             "Eval Time (s)": eval_time_str
         }
    finally:
        # 7. Clean up temporary files created in THIS iteration
        for path in [temp_orig_bea_path, temp_cor_bea_path, temp_hyp_m2_path]:
            if path and os.path.exists(path):
                try:
                    os.remove(path)
                except OSError as e:
                    print(f"  Error removing temp file {path}: {e}")

        # 8. Explicitly delete the LLM object to free VRAM
        if llm is not None:
            print(f"Unloading model: {llm.model_name}")
            del llm
            # Optional: Force garbage collection if memory issues persist
            # import gc
            # gc.collect()
            # If llama-cpp-python uses torch backend implicitly:
            # try:
            #     import torch
            #     torch.cuda.empty_cache()
            # except ImportError:
            #     pass # Ignore if torch isn't installed
        print("-" * 30)


# --- Final Cleanup for Subset Reference M2 ---
if temp_ref_m2_subset_path and os.path.exists(temp_ref_m2_subset_path):
     try:
         os.remove(temp_ref_m2_subset_path)
     except OSError as e:
         print(f"  Error removing {temp_ref_m2_subset_path}: {e}")



--- Starting Benchmark ---
Detokenizing 10 sentences for LLM input...
Detokenization complete.
Creating subset M2 file (10 sentences) at: /tmp/tmptxa2vhcg.m2
Finished creating subset M2 file.
Using subset reference M2: /tmp/tmptxa2vhcg.m2

===== Loading model: gemma-3-4b-it =====

Initializing gemma-3-4b-it-GGUF...
Using gec prompt: 


llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


gemma-3-4b-it-GGUF initialized.

===== Evaluating model: gemma-3-4b-it-GGUF =====
Created temp orig (BEA format): /tmp/tmptfjdatea.txt
Generating corrections for 10 sentences...
  Processing prompt 10/10...
  Finished processing 10 prompts.
Generation took: 5.26 seconds
Retokenizing LLM output to BEA format...
Retokenization complete.

--- Sample Outputs for gemma-3-4b-it-GGUF (First 10) ---
Orig (BEA)  [1]: It 's difficult answer at the question " what are you going to do in the future ? " if the only one who has to know it is in two minds .
Input (Nat) [1]: It's difficult answer at the question " what are you going to do in the future? " if the only one who has to know it is in two minds.
Output (Nat)[1]: It is difficult to answer the question "What are you going to do in the future?" when only one person is unsure.
Corr (BEA)  [1]: It is difficult to answer the question "What are you going to do in the future ?" when only one person is unsure .
---
Orig (BEA)  [2]: When I was younge

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


nomodit-4b-merged-v0-Q4_K_M-GGUF initialized.

===== Evaluating model: nomodit-4b-merged-v0-Q4_K_M-GGUF =====
Created temp orig (BEA format): /tmp/tmp3xve29gw.txt
Generating corrections for 10 sentences...
  Processing prompt 10/10...
  Finished processing 10 prompts.
Generation took: 5.01 seconds
Retokenizing LLM output to BEA format...
Retokenization complete.

--- Sample Outputs for nomodit-4b-merged-v0-Q4_K_M-GGUF (First 10) ---
Orig (BEA)  [1]: It 's difficult answer at the question " what are you going to do in the future ? " if the only one who has to know it is in two minds .
Input (Nat) [1]: It's difficult answer at the question " what are you going to do in the future? " if the only one who has to know it is in two minds.
Output (Nat)[1]: It's a difficult answer to the question "what are you going to do in the future?" if the only one who has to know it is in two minds.
Corr (BEA)  [1]: It 's a difficult answer to the question "what are you going to do in the future ?" if the

llama_init_from_model: n_ctx_per_seq (512) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


nomodit-4b-merged-Q4_K_M-GGUF initialized.

===== Evaluating model: nomodit-4b-merged-Q4_K_M-GGUF =====
Created temp orig (BEA format): /tmp/tmpvekg2up7.txt
Generating corrections for 10 sentences...
  Processing prompt 10/10...
  Finished processing 10 prompts.
Generation took: 5.12 seconds
Retokenizing LLM output to BEA format...
Retokenization complete.

--- Sample Outputs for nomodit-4b-merged-Q4_K_M-GGUF (First 10) ---
Orig (BEA)  [1]: It 's difficult answer at the question " what are you going to do in the future ? " if the only one who has to know it is in two minds .
Input (Nat) [1]: It's difficult answer at the question " what are you going to do in the future? " if the only one who has to know it is in two minds.
Output (Nat)[1]: It's a difficult answer to the question "What are you going to do in the future?" if the only one who has to know it is in two minds.
Corr (BEA)  [1]: It 's a difficult answer to the question "What are you going to do in the future ?" if the only one

In [25]:

print("\n--- Benchmark Results ---")
# Sort results alphabetically by model name for consistent output
sorted_results = dict(sorted(results.items()))

for model_name, scores in sorted_results.items():
    print(f"Model: {model_name}")
    # Define a preferred order for metrics if desired
    metric_order = ["F0.5", "P", "R", "Samples", "Generation Time (s)", "Eval Time (s)"]
    for metric in metric_order:
        if metric in scores:
            value = scores[metric]
            # Format floats nicely
            if isinstance(value, float):
                print(f"  {metric}: {value:.4f}")
            else:
                print(f"  {metric}: {value}")
    # Print any other metrics not in the preferred order (e.g., error messages)
    for metric, value in scores.items():
        if metric not in metric_order:
             print(f"  {metric}: {value}")
    print("-" * 20)


--- Benchmark Results ---
Model: gemma-3-4b-it-GGUF
  F0.5: 0.3106
  P: 0.2941
  R: 0.4000
  Samples: 10
  Generation Time (s): 5.26
  Eval Time (s): 2.68
--------------------
Model: nomodit-4b-merged-Q4_K_M-GGUF
  F0.5: 0.3763
  P: 0.4118
  R: 0.2800
  Samples: 10
  Generation Time (s): 5.12
  Eval Time (s): 1.95
--------------------
Model: nomodit-4b-merged-v0-Q4_K_M-GGUF
  F0.5: 0.4494
  P: 0.5000
  R: 0.3200
  Samples: 10
  Generation Time (s): 5.01
  Eval Time (s): 1.80
--------------------
